In [1]:
import torch

In [2]:
import matplotlib.pyplot as plt
import librosa
from torch import nn as nn
import torch.functional as F
import pandas as pd
import os
import numpy as np

In [5]:
def pad_layer(inp, layer, pad_type='reflect'):
    kernel_size = layer.kernel_size[0]
    if kernel_size % 2 == 0:
        pad = (kernel_size//2, kernel_size//2 - 1)
    else:
        pad = (kernel_size//2, kernel_size//2)
    # padding
    inp = F.pad(inp, 
            pad=pad,
            mode=pad_type)
    out = layer(inp)
    return out

In [6]:
def pad_layer_2d(inp, layer, pad_type='reflect'):
    kernel_size = layer.kernel_size
    if kernel_size[0] % 2 == 0:
        pad_lr = [kernel_size[0]//2, kernel_size[0]//2 - 1]
    else:
        pad_lr = [kernel_size[0]//2, kernel_size[0]//2]
    if kernel_size[1] % 2 == 0:
        pad_ud = [kernel_size[1]//2, kernel_size[1]//2 - 1]
    else:
        pad_ud = [kernel_size[1]//2, kernel_size[1]//2]
    pad = tuple(pad_lr + pad_ud)
    # padding
    inp = F.pad(inp, 
            pad=pad,
            mode=pad_type)
    out = layer(inp)
    return out

In [4]:
class SpeakerEncoder(nn.Module):
    def __init__(self, c_in, c_h, c_out,kernel_size
                bank_size, bank_scale, c_bank,
                n_conv_blocks, n_dense_blocks,
                ):
        super(SpeakerEncoder, self).__init__()
        self.c_in = c_in
        self.c_h = c_h
        self.c_out = c_out
        self.kernel_size = kernel_size
        self.n_conv_blocks = n_conv_blocks
        self.n_dense_blocks = n_dense_blocks
        self.subsample = subsample
        
        self.conv_bank = nn.ModuleList([nn.Conv1d(c_in, c_bank, kernel_size=k) for k in range(bank_scale, bank_size+1, bank_scale)])
        in_channels = c_bank*(bank_size//bank_size)+c_in
        self.in_conv_layer = nn.Conv1d()